In [5]:
import tensorflow as tf
import numpy as np
#tf.compat.v1.set_random_seed(2)
#np.random.seed(2)

def add_layer(inputs, input_size, output_size, activation_function=None):
    Weights = tf.Variable(tf.random_uniform([input_size, output_size], minval=-0.05, maxval=0.05))
    threshold = tf.Variable(tf.zeros([1, output_size]) + 1)
    Wx_plus_b = tf.matmul(inputs, Weights) + threshold
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs, Weights, threshold

def nStage(x_data, y_data, n, inputSize):
    tmp = np.full((n, inputSize), 0.00)
    tmp2 = np.full((n, 1), 0.00)           
    for i in range(n):
        tmp[i] = x_data[i]
        tmp2[i] = y_data[i]
        
    a_data = []
    b_data = []
    for i in range(n):
        if tmp2[i] > 0:
            a_data.append(i)
        else:
            b_data.append(i)
    return tmp, tmp2, a_data, b_data 

def LTS_sort(error, x_data, y_data, n):
    for i in range(1, n):
        tmp = np.array(error[i])
        tmp2 = np.array(x_data[i])
        tmp3 = np.array(y_data[i])
        j = i
        while j > 0 and tmp < error[j - 1]:
            error[j] = error[j - 1]
            x_data[j] = x_data[j - 1]
            y_data[j] = y_data[j - 1]
            j-=1   
        error[j] = tmp
        x_data[j] = tmp2
        y_data[j] = tmp3

    if y_data[0] == y_data[1]:
        for i in range(n - 2):
            if y_data[0] != y_data[i + 2]:
                tmp = np.array(x_data[1])
                tmp2 = np.array(y_data[1])
                x_data[1] = x_data[i + 2]
                y_data[1] = y_data[i + 2]
                x_data[i + 2] = tmp
                y_data[i + 2] = tmp2
                break
    return x_data, y_data


In [6]:
N = 60
inputSize = 22
fp = open("SPECT Heart train x_data.txt", "r", encoding = "utf-8")
x_data = np.full((N, inputSize), 0.00)

for i in range(N):
    line = fp.readline()
    for j in range(inputSize):
        x_data[i][j] = np.asarray(line.split(), dtype=np.float64)[j]
fp.close()

fp = open("SPECT Heart train y_data.txt", "r")
line = fp.readline()
y_data = []

while line:
    y_data.append(line.strip())
    line = fp.readline()
y_data = np.asarray(y_data, dtype=np.int32)
y_data = np.reshape(y_data, (N, -1))
from sklearn.utils import shuffle
x_data, y_data = shuffle(x_data, y_data, random_state=0)

a = -1
b = 1

for i in range(N): 
    if y_data[i] > 0:
        a_index = i
        break
for i in range(N): 
    if y_data[i] < 0:
        b_index = i
        break
    
nStage_x_data = np.full((2, inputSize), 0.00)
nStage_x_data[0] = x_data[a_index]
nStage_x_data[1] = x_data[b_index]
nStage_y_data = np.full((2, 1), 0.00)
nStage_y_data[0] = y_data[a_index]
nStage_y_data[1] = y_data[b_index]
a_data = [0]
b_data = [1]

xs = tf.placeholder(tf.float32, [None, inputSize])
ys = tf.placeholder(tf.float32, [None, 1])
learning_rate = tf.placeholder(tf.float32, shape=[])

In [7]:
hiddenLayer, weights_h, threshold_h = add_layer(xs, inputSize, 1, tf.nn.relu)
outputLayer, weights_o, threshold_o = add_layer(hiddenLayer, 1, 1)
hiddenNodesNum = 1

error = tf.reduce_mean(tf.reduce_sum(tf.square(ys - outputLayer), reduction_indices=[1]))
#regularizers = tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(threshold_h) + tf.nn.l2_loss(threshold_o)
regularizers = tf.math.multiply((0.001/(hiddenNodesNum + 1 + hiddenNodesNum * (inputSize + 1))), tf.reduce_sum(tf.square(weights_o))+tf.reduce_sum(tf.square(weights_h))+tf.reduce_sum(tf.square(threshold_h))+tf.reduce_sum(tf.square(threshold_o)))
error = error + regularizers
error2 = tf.square(ys - outputLayer)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess = tf.Session()
sess.run(init)

In [8]:
#Step1:　Two cases + Learning goal + one hidden node
lr = 0.1
while a <= b:
    sess.run(train, feed_dict={xs: nStage_x_data, ys: nStage_y_data, learning_rate: lr})
    pred_ = sess.run(outputLayer, feed_dict={xs: nStage_x_data, ys: nStage_y_data, learning_rate: lr})
    a = min(pred_[a_data])
    b = max(pred_[b_data])
print("Step1: ")
print("a: ", a)
print("b: ", b)
    
learning_times = 1
n = 3
ud = False

#Step2: n > N?
while n <= N:
    print("stage", n, "/", N, ":")
    #Step3: LTS
    error_ = sess.run(error2, feed_dict={xs: x_data, ys: y_data})
    x_data, y_data = LTS_sort(error_, x_data, y_data, N)
    nStage_x_data, nStage_y_data, a_data, b_data = nStage(x_data, y_data, n, inputSize)
    
    pred_, temp_loss, temp_weights_h, temp_weights_o, temp_threshold_h, temp_threshold_o = sess.run([outputLayer, error, weights_h, weights_o, threshold_h, threshold_o], feed_dict={xs: nStage_x_data, ys: nStage_y_data})
    a = min(pred_[a_data])
    b = max(pred_[b_data])
    lr = 0.1
    learning_times = 0
    
    #Step4: Learning goal?
    while a <= b:
        #Step5: Save w
        n1Stage_a = a
        n1Stage_b = b
        n1Stage_nCase = pred_[n - 1]
        n1Stage_weights_h, n1Stage_weights_o, n1Stage_threshold_h, n1Stage_threshold_o = sess.run([weights_h, weights_o, threshold_h, threshold_o])
        
        #Step6: Weight-tuning + Cramming
        sess.run(train, feed_dict={xs: nStage_x_data, ys: nStage_y_data, learning_rate: lr})
        pred_, loss = sess.run([outputLayer, error], feed_dict={xs: nStage_x_data, ys: nStage_y_data})
        if loss < temp_loss:
            lr = lr * 1.2
            a = min(pred_[a_data])
            b = max(pred_[b_data])
            temp_weights_h, temp_weights_o, temp_threshold_h, temp_threshold_o = sess.run([weights_h, weights_o, threshold_h, threshold_o])
            temp_loss = loss
            learning_times += 1
            if learning_times % 10 == 0:
                print("   loss: ", loss)
        else:
            if  lr > 0.05:      
                sess.run(weights_h.assign(temp_weights_h))
                sess.run(weights_o.assign(temp_weights_o))
                sess.run(threshold_h.assign(temp_threshold_h))
                sess.run(threshold_o.assign(temp_threshold_o))
                lr = lr * 0.7
            else:
                print("   undesired attractor")
                #新增 hidden nodes 並將新增的 weights 權重設為0，新神經網路的output必須與原砷經網路相同
                #error、init 需重新指派
                hiddenNodesNum += 1                 
                hiddenLayer, weights_h, threshold_h = add_layer(xs, inputSize, hiddenNodesNum, tf.nn.relu)
                outputLayer, weights_o, threshold_o = add_layer(hiddenLayer, hiddenNodesNum, 1)
                error = tf.reduce_mean(tf.reduce_sum(tf.square(ys - outputLayer), reduction_indices=[1]))
                #regularizers = tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(threshold_h) + tf.nn.l2_loss(threshold_o)
                regularizers = tf.math.multiply((0.001/(hiddenNodesNum + 1 + hiddenNodesNum * (inputSize + 1))), tf.reduce_sum(tf.square(weights_o))+tf.reduce_sum(tf.square(weights_h))+tf.reduce_sum(tf.square(threshold_h))+tf.reduce_sum(tf.square(threshold_o)))
                error = error + regularizers
                error2 = tf.square(ys - outputLayer)
                train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

                init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
                sess = tf.Session()
                sess.run(init)

                J = 1.1 #滔
                new_weights_h = np.hstack([n1Stage_weights_h, np.vstack([J * nStage_x_data[n - 1][0], J * nStage_x_data[n - 1][1], J * nStage_x_data[n - 1][2], J * nStage_x_data[n - 1][3], J * nStage_x_data[n - 1][4], J * nStage_x_data[n - 1][5], J * nStage_x_data[n - 1][6], J * nStage_x_data[n - 1][7], J * nStage_x_data[n - 1][8], J * nStage_x_data[n - 1][9], J * nStage_x_data[n - 1][10], J * nStage_x_data[n - 1][11], J * nStage_x_data[n - 1][12], J * nStage_x_data[n - 1][13], J * nStage_x_data[n - 1][14], J * nStage_x_data[n - 1][15], J * nStage_x_data[n - 1][16], J * nStage_x_data[n - 1][17], J * nStage_x_data[n - 1][18], J * nStage_x_data[n - 1][19], J * nStage_x_data[n - 1][20], J * nStage_x_data[n - 1][21]])])
                new_threshold_h = np.full((1, 1), (1 - inputSize) * J)
                new_threshold_h = np.hstack([n1Stage_threshold_h, new_threshold_h])
                if nStage_y_data[n - 1] > 0:
                    new_weights_o = n1Stage_b - n1Stage_nCase
                    if new_weights_o == 0:
                        new_weights_o = 0.1
                else:
                    new_weights_o = n1Stage_a - n1Stage_nCase
                    if new_weights_o == 0: 
                        new_weights_o = -0.1
                new_weights_o = np.vstack([n1Stage_weights_o, new_weights_o]) 
                sess.run(weights_h.assign(new_weights_h))
                sess.run(weights_o.assign(new_weights_o))
                sess.run(threshold_h.assign(new_threshold_h))
                sess.run(threshold_o.assign(n1Stage_threshold_o))

                pred_ = sess.run(outputLayer, feed_dict={xs: nStage_x_data})
                a = min(pred_[a_data])
                b = max(pred_[b_data])
                if(a > b):
                    print("   cramming succes")
                else:
                    print("   cramming error, learning break")
                    ud = True                    
                break
    print("   learning_times:", learning_times)
    if ud: break

    if hiddenNodesNum != 1:
        #Step7: The Softening and Integrating mechanism
        hiddenNodesIndex = 1
        while hiddenNodesIndex <= hiddenNodesNum:
            print("   Sof&Int: ", hiddenNodesIndex, "/", hiddenNodesNum)
            #Softening
            sftTimes = 0
            lr = 0.1
            pred_, temp_loss, temp_weights_h, temp_weights_o, temp_threshold_h, temp_threshold_o = sess.run([outputLayer, error, weights_h, weights_o, threshold_h, threshold_o], feed_dict={xs: nStage_x_data, ys: nStage_y_data})
            a = min(pred_[a_data])
            b = max(pred_[b_data])
            while a > b and sftTimes < 500:
                pre_weights_h, pre_weights_o, pre_threshold_h, pre_threshold_o = sess.run([weights_h, weights_o, threshold_h, threshold_o])
                sess.run(train, feed_dict={xs: nStage_x_data, ys: nStage_y_data, learning_rate: lr})
                pred_, loss = sess.run([outputLayer, error], feed_dict={xs: nStage_x_data, ys: nStage_y_data})             
                if loss < temp_loss:
                    lr = lr * 1.2
                    a = min(pred_[a_data])
                    b = max(pred_[b_data])
                    temp_weights_h, temp_weights_o, temp_threshold_h, temp_threshold_o = sess.run([weights_h, weights_o, threshold_h, threshold_o])
                    temp_loss = loss
                    sftTimes += 1
                    if sftTimes % 100 == 0:
                        print("   S_loss: ", loss)
                        print("   r: ", sess.run(regularizers))
                        print("   hidd: ", sess.run(hiddenLayer, feed_dict={xs: nStage_x_data}))
                        print("   weight_h: ", sess.run(weights_h))
                else:
                    if  lr > 0.05:
                        sess.run(weights_h.assign(temp_weights_h))
                        sess.run(weights_o.assign(temp_weights_o))
                        sess.run(threshold_h.assign(temp_threshold_h))
                        sess.run(threshold_o.assign(temp_threshold_o))
                        lr = lr * 0.7
                    else:
                        break;     
            sess.run(weights_h.assign(pre_weights_h))
            sess.run(weights_o.assign(pre_weights_o))
            sess.run(threshold_h.assign(pre_threshold_h))
            sess.run(threshold_o.assign(pre_threshold_o))           
            print("   Softening times:", sftTimes)

            pre_weights_h, pre_weights_o, pre_threshold_h, pre_threshold_o = sess.run([weights_h, weights_o, threshold_h, threshold_o])
            #暫時忽略一個 hidden node
            #error、init 需重新指派
            hiddenNodesNum -= 1                 
            hiddenLayer, weights_h, threshold_h = add_layer(xs, inputSize, hiddenNodesNum, tf.nn.relu)
            outputLayer, weights_o, threshold_o = add_layer(hiddenLayer, hiddenNodesNum, 1)
            error = tf.reduce_mean(tf.reduce_sum(tf.square(ys - outputLayer), reduction_indices=[1]))
            #regularizers = tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(threshold_h) + tf.nn.l2_loss(threshold_o)
            regularizers = tf.math.multiply((0.001/(hiddenNodesNum + 1 + hiddenNodesNum * (inputSize + 1))), tf.reduce_sum(tf.square(weights_o))+tf.reduce_sum(tf.square(weights_h))+tf.reduce_sum(tf.square(threshold_h))+tf.reduce_sum(tf.square(threshold_o)))
            error = error + regularizers
            error2 = tf.square(ys - outputLayer)
            train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

            init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
            sess = tf.Session()
            sess.run(init)

            sess.run(weights_h.assign(np.delete(pre_weights_h, hiddenNodesIndex - 1, 1)))
            sess.run(weights_o.assign(np.delete(pre_weights_o, hiddenNodesIndex - 1, 0)))
            sess.run(threshold_h.assign(np.delete(pre_threshold_h, hiddenNodesIndex - 1, 1)))
            sess.run(threshold_o.assign(pre_threshold_o))

            pred_, temp_loss, temp_weights_h, temp_weights_o, temp_threshold_h, temp_threshold_o = sess.run([outputLayer, error, weights_h, weights_o, threshold_h, threshold_o], feed_dict={xs: nStage_x_data, ys: nStage_y_data})
            a = min(pred_[a_data])
            b = max(pred_[b_data])
            lr = 0.1
            isIntegrating = True
            iTimes = 0
            while a <= b:
                if iTimes >= 200:
                    isIntegrating = False           
                    break

                sess.run(train, feed_dict={xs: nStage_x_data, ys: nStage_y_data, learning_rate: lr})
                pred_, loss = sess.run([outputLayer, error], feed_dict={xs: nStage_x_data, ys: nStage_y_data})
                if loss < temp_loss:
                    lr = lr * 1.2
                    a = min(pred_[a_data])
                    b = max(pred_[b_data])
                    temp_weights_h, temp_weights_o, temp_threshold_h, temp_threshold_o = sess.run([weights_h, weights_o, threshold_h, threshold_o])
                    temp_loss = loss
                    iTimes += 1
                    if iTimes % 100 == 0:
                        print("   I_loss: ", loss)
                else:
                    if  lr > 0.05:      
                        sess.run(weights_h.assign(temp_weights_h))
                        sess.run(weights_o.assign(temp_weights_o))
                        sess.run(threshold_h.assign(temp_threshold_h))
                        sess.run(threshold_o.assign(temp_threshold_o))
                        lr = lr * 0.7
                    else:
                        isIntegrating = False           
                        break
            if isIntegrating: 
                print("   iTimes:", iTimes, ", Integrating sucess")
            else:
                #回復hidden node
                hiddenNodesNum += 1                 
                hiddenLayer, weights_h, threshold_h = add_layer(xs, inputSize, hiddenNodesNum, tf.nn.relu)
                outputLayer, weights_o, threshold_o = add_layer(hiddenLayer, hiddenNodesNum, 1)
                error = tf.reduce_mean(tf.reduce_sum(tf.square(ys - outputLayer), reduction_indices=[1]))
                #regularizers = tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_o) + tf.nn.l2_loss(threshold_h) + tf.nn.l2_loss(threshold_o)
                regularizers = tf.math.multiply((0.001/(hiddenNodesNum + 1 + hiddenNodesNum * (inputSize + 1))), tf.reduce_sum(tf.square(weights_o))+tf.reduce_sum(tf.square(weights_h))+tf.reduce_sum(tf.square(threshold_h))+tf.reduce_sum(tf.square(threshold_o)))
                error = error + regularizers
                error2 = tf.square(ys - outputLayer)
                train = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

                init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
                sess = tf.Session()
                sess.run(init)

                sess.run(weights_h.assign(pre_weights_h))
                sess.run(weights_o.assign(pre_weights_o))
                sess.run(threshold_h.assign(pre_threshold_h))
                sess.run(threshold_o.assign(pre_threshold_o))
                print("   iTimes:", iTimes, ", Integrating fail, restore weight")
            hiddenNodesIndex += 1          
    n+=1

print("Complete!")

test_N = 174
fp = open("test_x_data.txt", "r", encoding = "utf-8")
test_x_data = np.full((test_N, inputSize), 0.00)

for i in range(test_N):
    line = fp.readline()
    for j in range(inputSize):
        test_x_data[i][j] = np.asarray(line.split(), dtype=np.float64)[j]
fp.close()

fp = open("test_y_data.txt", "r")
line = fp.readline()
test_y_data = []

while line:
    test_y_data.append(line.strip())
    line = fp.readline()
test_y_data = np.asarray(test_y_data, dtype=np.int32)
test_y_data = np.reshape(test_y_data, (test_N, -1))

pred_ = sess.run(outputLayer, feed_dict={xs: x_data})
a = min(pred_[a_data])
b = max(pred_[b_data])
print("a: ", a)
print("b: ", b)

raw_pred = sess.run(outputLayer, feed_dict={xs: test_x_data})
pred = []
for i in range(test_N):        
    if raw_pred[i] >= a:
        pred.append(1)
    elif raw_pred[i] <= b:
        pred.append(-1)
    else:
        pred.append(0)
pred = np.asarray(pred, dtype=np.int32)
pred = np.reshape(pred, (test_N, -1))

cpred = tf.equal(pred, test_y_data)
acc = tf.reduce_mean(tf.cast(cpred, tf.float32))
print("accuracy: ", sess.run(acc))

sess.close()


Step1: 
a:  [0.5957364]
b:  [0.5588237]
stage 3 / 60 :
   learning_times: 0
stage 4 / 60 :
   learning_times: 0
stage 5 / 60 :
   learning_times: 0
stage 6 / 60 :
   learning_times: 0
stage 7 / 60 :
   learning_times: 0
stage 8 / 60 :
   learning_times: 0
stage 9 / 60 :
   learning_times: 0
stage 10 / 60 :
   learning_times: 0
stage 11 / 60 :
   learning_times: 0
stage 12 / 60 :
   learning_times: 0
stage 13 / 60 :
   learning_times: 0
stage 14 / 60 :
   learning_times: 0
stage 15 / 60 :
   learning_times: 0
stage 16 / 60 :
   learning_times: 0
stage 17 / 60 :
   learning_times: 0
stage 18 / 60 :
   learning_times: 0
stage 19 / 60 :
   learning_times: 0
stage 20 / 60 :
   learning_times: 0
stage 21 / 60 :
   learning_times: 0
stage 22 / 60 :
   learning_times: 0
stage 23 / 60 :
   learning_times: 0
stage 24 / 60 :
   learning_times: 0
stage 25 / 60 :
   learning_times: 0
stage 26 / 60 :
   learning_times: 0
stage 27 / 60 :
   learning_times: 0
stage 28 / 60 :
   learning_times: 0
stage

KeyboardInterrupt: 